In [ ]:
from PyPDF2 import PdfReader

In [ ]:
reader = PdfReader("ppk/özet/2024/DUY2024-03.pdf")
number_of_pages = len(reader.pages)
print("number of pages:", number_of_pages)
text = ""
for i in range(number_of_pages):
    page = reader.pages[i]
    text += page.extract_text()

In [ ]:
res = re.findall(r"((^|\n|\s)(\d){1,2}\s\n\s)", text)
print(res)

In [ ]:
def _process_doc(doc_text, no_of_pages, doc_name=None):
    text = doc_text

    regex2 = r"\s\(:\(:\):\)\s"

    

    # discard footnote references

    re_1 = r"(([a-zğüşıöçâîû])(\.\d|\d\.)\s)"
    res_1 = re.findall(re_1, text)
    print(res_1)
    text = re.sub(re_1, "\\2. ", text)

    # discard footnotes

    if len(res_1) > 0:    
        re_0 = r"((\s){2,}[1-9][0-9]?\s[A-Z0-9ĞÜŞİÖÇÂÎÛ:,;\-’\.“”\(\)][a-zA-Z0-9ğüşıöçĞÜŞİÖÇÂâÎîÛû:,\-’\.“”\(\)\s]{50,}(\s([2-9]|[1-9][0-9])\s\s))"
        res_0 = re.findall(re_0, text)
        print(res_0)
        text = re.sub(re_0, "\\3", text)
    

    
    
    # parse document

    regex1 = r"([1-9][0-9]?\.\s)"
    regex3 = r"(Sayı: \d\d\d\d-\d\d)"
    regex4 = r"((Para Politikası Kurulu Toplantı Özeti)|(PARA POLİTİKASI KURULU TOPLANTI ÖZETİ))"
    regex5 = r"(Toplantı Tarihi: \d\d? [a-zA-ZşığüŞİĞÜ]{4,7} \d\d\d\d)" 
    regex6 = r"(\d\d? [a-zA-ZşığüŞİĞÜ]{4,7} \d\d\d\d)"
    regex7 = r"BASIN DUYURUSU"
    regex8 = r"([a-zA-Z0-9ğüşıöçĞÜŞİÖÇÂâÎîÛû:,;\-/'’“”\(\)\s]{2,}\.)"

    res = re.findall(regex1, text)
    # print(res)
    new_text = re.sub(regex1, regex2 + "\\1", text)
    new_text_parts = new_text.split(regex2)
    parsed = {
        "number": "",
        "date": "",
        "published": "",
        "topics": []
    }
    paragraph = ""
    paragraph_items = []
    for i, p in enumerate(new_text_parts):
        res2 = re.findall(regex1, p)
        if len(res2) == 0:
            res_0_0 = re.findall(regex3, p)
            res_0_1 = re.findall(regex4, p)
            res_0_2 = re.findall(regex5, p)
            res_0_3 = re.findall(regex7, p)
            # print(res_0_0)
            # print(res_0_1)
            # print(res_0_2)
            parsed["number"] = res_0_0[0].split(":")[1].strip()
            parsed["date"] = res_0_2[0].split(":")[1].strip()
            p = re.sub(regex3, "", p)
            p = re.sub(regex4, "", p)
            p = re.sub(regex5, "", p)
            p = re.sub(regex7, "", p)
            res_0_4 = re.findall(regex6, p)
            # print(res_0_4)
            parsed["published"] = res_0_4[0]
            p = re.sub(regex6, "", p)
            p = p.replace(",", "")
            paragraph = p.strip()

        elif len(res2) == 1:
            p = re.sub(regex1, "", p)
            res3 = re.findall(regex8, p)
            p = re.sub(regex8, "", p)
            # print(p)
            item_idx = int(res2[0].split(".")[0])
            item_text = " ".join([r.strip() for r in res3])
            paragraph_items.append({"index": item_idx, "text": item_text})
            if p.strip() == "":
                if i == len(new_text_parts) - 1:
                    parsed["topics"].append({"title": paragraph, "items": paragraph_items})
            else:
                parsed["topics"].append({"title": paragraph, "items": paragraph_items})
                paragraph = p.strip()
                paragraph_items = []

        else:
            print("error")

    return parsed